<a href="https://colab.research.google.com/github/MenaHagag/Capstone-Project-/blob/master/The_Battle_of_the_Neighborhood(Week2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project-The Battle of the Neighborhood(Week2)

#Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Middel Eastern restaurant** in **Toronto, Canada.**

We are interested examing the City of Toronto. We will target Toronto downtown. Since there are lots of restaurants in Toronto's downtown, we will try to find the location with **the least number of middle eastern restuarants and closer to downtown.**

We will use data science to examine Toronto's downtown and search for the best location to be chosen by stackholders.



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # beautifulsoup
import requests
print('Libraries imported.')

Libraries imported.


# Data


In this section we will extract and prep the data.

In [0]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text



In [0]:
# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [0]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

 #print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        if (Postcode_var == 'Not assigned\n' or Borough_var == 'Not assigned\n' or Neighbourhood_var == 'Not assigned\n'):
            continue
        if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
            continue
        
        Postcode.append(Postcode_var[:-1])
        Borough.append(Borough_var[:-1])
        Neighbourhood.append(Neighbourhood_var)
   

In [5]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 103


In [0]:
# Create a dataframe that consist of the of three columns: PostalCode, Borough, and Neighborhood

In [7]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(11)

,Postcode,Borough,Neighbourhood
0,M5N,Central Toronto,Roselawn
1,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
2,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
3,M5M,North York,"Bedford Park, Lawrence Manor East"
4,M4S,Central Toronto,Davisville
5,M7R,Mississauga,Canada Post Gateway Processing Centre
6,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
7,M5E,Downtown Toronto,Berczy Park
8,M5W,Downtown Toronto,Stn A PO Boxes
9,M8W,Etobicoke,"Alderwood, Long Branch"


In [8]:
df_toronto.shape

(103, 3)

Extract latidutes and longitudes from italicized text "Geospatial Coordinates.csv" file and append them accordingly to above dataframe.

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [10]:
df_GC = pd.read_csv("Geospatial_Coordinates.csv")
df_GC.head(11)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
df_GC.shape

(103, 3)

In [12]:
df_toronto.head(5)

,Postcode,Borough,Neighbourhood
0,M5N,Central Toronto,Roselawn
1,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
2,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
3,M5M,North York,"Bedford Park, Lawrence Manor East"
4,M4S,Central Toronto,Davisville


In [0]:
df_toronto['Latitude'] = pd.Series([0.00 for x in range(len(df_toronto.index))])

In [14]:
df_toronto['Longitude'] = pd.Series([0.00 for x in range(len(df_toronto.index))])

df_toronto.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,0.0,0.0
1,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",0.0,0.0
2,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",0.0,0.0
3,M5M,North York,"Bedford Park, Lawrence Manor East",0.0,0.0
4,M4S,Central Toronto,Davisville,0.0,0.0


In [15]:
for i in range(len(df_toronto.index)):
  ps = df_toronto["Postcode"][i]
  for g in range(len(df_GC.index)):
    psg = df_GC["Postal Code"][g]
    if ps == psg : 
      df_toronto['Latitude'][i] = df_GC['Latitude'][g]
      df_toronto['Longitude'][i] = df_GC['Longitude'][g]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Our new dataframe with latitude and longitude appeneded.

In [16]:
df_toronto.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577
2,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
3,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
4,M4S,Central Toronto,Davisville,43.704324,-79.388790
5,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
6,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
8,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846
9,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [17]:
# toronto
df_borough = df_toronto[df_toronto['Borough'].str.find('Toronto')>-1]
df_borough.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
4,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
8,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846
11,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [0]:
# Downtown Toronto
df_downtown_toronto = df_borough[df_borough['Borough'].str.find('Downtown Toronto')>-1]
df_downtown_toronto.head(5)

Queens  = df_borough[df_borough['Neighbourhood'].str.find('Queen\'s Park, Ontario Provincial Government')>-1]
Queens = Queens.reset_index()

In [19]:
# Queen's Park

#reset index to 0
downtown_latitude = Queens['Latitude'][0]
downtown_longitude = Queens['Longitude'][0]
print('Coordinate of Queen\'s Park [ ',downtown_latitude, ',' , downtown_longitude,'] ')

Coordinate of Queen's Park [  43.6623015 , -79.3894938 ] 


In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [21]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

     |████████████████████████████████| 10.9MB 2.7MB/s 


In [22]:


# changing the coordinates from degress to meters

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)


print('Coordinate transformation check')
print('-------------------------------')
print('Downtown Toronto  latitude={}, longitude={}'.format(downtown_latitude, downtown_longitude))
x, y = lonlat_to_xy(downtown_longitude, downtown_latitude)
print('Downtown Toronto UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Downtown Toronto  latitude={}, longitude={}'.format(la, lo))


Coordinate transformation check
-------------------------------
Downtown Toronto  latitude=43.6623015, longitude=-79.3894938
Downtown Toronto UTM X=-5309052.987990187, Y=10508025.230640132
Downtown Toronto  latitude=43.66230149999975, longitude=-79.38949380000048


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [23]:
downtown_toronto_x, downtown_toronto_y = lonlat_to_xy(downtown_longitude, downtown_latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = downtown_toronto_x - 6000
x_step = 600
y_min = downtown_toronto_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(downtown_toronto_x, downtown_toronto_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [24]:
import pandas as pd

df_locations = pd.DataFrame({'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Latitude,Longitude,X,Y,Distance from center
0,43.655033,-79.338910,-5.310853e+06,1.050231e+07,5992.495307
1,43.658759,-79.339472,-5.310253e+06,1.050231e+07,5840.376700
2,43.662485,-79.340034,-5.309653e+06,1.050231e+07,5747.173218
3,43.666211,-79.340596,-5.309053e+06,1.050231e+07,5715.767665
4,43.669938,-79.341158,-5.308453e+06,1.050231e+07,5747.173218
5,43.673665,-79.341721,-5.307853e+06,1.050231e+07,5840.376700
6,43.677392,-79.342283,-5.307253e+06,1.050231e+07,5992.495307
7,43.649091,-79.342510,-5.311753e+06,1.050283e+07,5855.766389
8,43.652816,-79.343073,-5.311153e+06,1.050283e+07,5604.462508
9,43.656542,-79.343635,-5.310553e+06,1.050283e+07,5408.326913


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [25]:
down_town_cor = [downtown_latitude,downtown_longitude]
map_downtown_toronto = folium.Map(location=[downtown_latitude, downtown_longitude], zoom_start=13)
folium.Marker(down_town_cor, popup='Queen\'s Park').add_to(map_downtown_toronto)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=200, color='blue', fill=False).add_to(map_downtown_toronto)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_downtown_toronto

# Foursqaure



In [0]:
# Category IDs corresponding to Middle Eastern restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259'

middle_eastern_restaurant_categories = ['4bf58dd8d48988d115941735','5bae9231bedf3950379f89e1', '5bae9231bedf3950379f89e7',
'58daa1558bbb0b01f18ec1cd','5bae9231bedf3950379f89e4', '5bae9231bedf3950379f89da', '5bae9231bedf3950379f89ea']


In [0]:

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Toronto', '')
    address = address.replace(', Canada', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
      venues = []
    return venues

In [28]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle
foursquare_client_id = 'HEGMOCWBKFI4UO2A0PVCDZWO0NYKUIOJJP3DWVS5SLBC05OF'
foursquare_client_secret = 'OQESDM4L5AFXBFCDSHHMPBNFJXJFCWEG2F2FLWO3RBQZLTST'


def get_restaurants(lats, lons):
    restaurants = {}
    middle_eastern_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_middle_eastern = is_restaurant(venue_categories, specific_filter=middle_eastern_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_middle_eastern, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_middle_eastern:
                    middle_eastern_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, middle_eastern_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
middle_eastern_restaurants = {}
location_restaurants = []
loaded = False
'''try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('middle_eastern_restaurants_350.pkl', 'rb') as f:
        middle_eastern_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass
'''
# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, middle_eastern_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('middle_eastern_restaurants_350.pkl', 'wb') as f:
        pickle.dump(middle_eastern_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [29]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Middle Eastern restaurants:', len(middle_eastern_restaurants))
print('Percentage of Middle Eastern restaurants: {:.2f}%'.format(len(middle_eastern_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1280
Total number of Middle Eastern restaurants: 43
Percentage of Middle Eastern restaurants: 3.36%
Average number of restaurants in neighborhood: 5.645604395604396


Let's now see all the collected restaurants in our area of interest on map, and let's also show Middle Eastern restaurants in different color.

In [30]:
map_toronto = folium.Map(location= [downtown_latitude, downtown_longitude], zoom_start=13)
folium.Marker(down_town_cor, popup='Queen\'s Park').add_to(map_toronto)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_middle_eastern = res[6]
    color = 'red' if is_middle_eastern else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
map_toronto

#Analysis

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [31]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 5.645604395604396


,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,43.655033,-79.338910,-5.310853e+06,1.050231e+07,5992.495307,2
1,43.658759,-79.339472,-5.310253e+06,1.050231e+07,5840.376700,5
2,43.662485,-79.340034,-5.309653e+06,1.050231e+07,5747.173218,5
3,43.666211,-79.340596,-5.309053e+06,1.050231e+07,5715.767665,4
4,43.669938,-79.341158,-5.308453e+06,1.050231e+07,5747.173218,3
5,43.673665,-79.341721,-5.307853e+06,1.050231e+07,5840.376700,1
6,43.677392,-79.342283,-5.307253e+06,1.050231e+07,5992.495307,8
7,43.649091,-79.342510,-5.311753e+06,1.050283e+07,5855.766389,0
8,43.652816,-79.343073,-5.311153e+06,1.050283e+07,5604.462508,0
9,43.656542,-79.343635,-5.310553e+06,1.050283e+07,5408.326913,2


OK, now let's calculate the distance to nearest Middle Eastern restaurant from every area candidate center (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [0]:
distances_to_middle_eastern_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in middle_eastern_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_middle_eastern_restaurant.append(min_distance)

df_locations['Distance to Middele_Eastern restaurant'] = distances_to_middle_eastern_restaurant

In [33]:
df_locations.head(10)

,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Middele_Eastern restaurant
0,43.655033,-79.338910,-5.310853e+06,1.050231e+07,5992.495307,2,1145.187917
1,43.658759,-79.339472,-5.310253e+06,1.050231e+07,5840.376700,5,813.294587
2,43.662485,-79.340034,-5.309653e+06,1.050231e+07,5747.173218,5,855.243126
3,43.666211,-79.340596,-5.309053e+06,1.050231e+07,5715.767665,4,1233.464034
4,43.669938,-79.341158,-5.308453e+06,1.050231e+07,5747.173218,3,1741.099148
5,43.673665,-79.341721,-5.307853e+06,1.050231e+07,5840.376700,1,2293.778316
6,43.677392,-79.342283,-5.307253e+06,1.050231e+07,5992.495307,8,2865.556086
7,43.649091,-79.342510,-5.311753e+06,1.050283e+07,5855.766389,0,1760.523646
8,43.652816,-79.343073,-5.311153e+06,1.050283e+07,5604.462508,0,1170.229135
9,43.656542,-79.343635,-5.310553e+06,1.050283e+07,5408.326913,2,599.523934


In [34]:
print('Average distance to closest Middle_Eastern restaurant from each area center:', df_locations['Distance to Middele_Eastern restaurant'].mean())

Average distance to closest Middle_Eastern restaurant from each area center: 1218.0841989005698


#Clustering 

In [35]:
#Finding the good location(contains lower number of middle eastern restaurant and other different resturants  )

good_res_count = np.array((df_locations['Restaurants in area']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_me_distance = np.array(df_locations['Distance to Middele_Eastern restaurant']>=400)
print('Locations with no  Middele_Eastern restaurants within 400m:', good_me_distance.sum())

good_locations = np.logical_and(good_res_count, good_me_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]

Locations with no more than two restaurants nearby: 188
Locations with no  Middele_Eastern restaurants within 400m: 316
Locations with both conditions met: 187


In [0]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

Run *k*-means to cluster the neighborhood into 10 clusters.

In [37]:
# set number of cluster

kclusters = 10

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(good_xys)

# lets a dataframe that includes the cluster 
cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_toronto_res = folium.Map(location=[downtown_latitude, downtown_longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_toronto_res)

folium.Marker(down_town_cor).add_to(map_toronto_res)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_toronto_res) 

map_toronto_res

In [38]:
kmeans.labels_

array([8, 1, 8, 8, 8, 1, 1, 1, 4, 8, 8, 8, 8, 1, 1, 4, 4, 8, 8, 8, 8, 1,
       1, 4, 4, 4, 8, 8, 8, 8, 8, 1, 1, 1, 1, 4, 4, 4, 4, 5, 5, 8, 8, 1,
       1, 1, 1, 4, 4, 4, 4, 4, 2, 5, 5, 5, 5, 8, 1, 4, 4, 4, 4, 2, 2, 2,
       5, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 5, 5, 5, 5, 4, 2, 2, 2, 2, 2, 5,
       5, 5, 9, 2, 2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 5, 5, 5, 9, 2, 3, 3,
       3, 9, 9, 6, 6, 6, 3, 3, 3, 3, 9, 9, 6, 6, 6, 6, 3, 3, 3, 9, 6, 6,
       6, 6, 6, 3, 3, 3, 3, 9, 6, 6, 6, 6, 3, 3, 3, 3, 7, 0, 6, 6, 6, 3,
       3, 3, 7, 7, 0, 0, 0, 6, 6, 3, 7, 0, 0, 0, 0, 6, 3, 7, 0, 0, 0, 0,
       0, 0, 0, 7, 0, 0, 0, 0, 7, 0, 0], dtype=int32)

In [39]:
df_good_locations['cluster_labels'] = kmeans.labels_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_good_locations.head(10)

,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Middele_Eastern restaurant,cluster_labels
0,43.655033,-79.338910,-5.310853e+06,1.050231e+07,5992.495307,2,1145.187917,8
5,43.673665,-79.341721,-5.307853e+06,1.050231e+07,5840.376700,1,2293.778316,1
7,43.649091,-79.342510,-5.311753e+06,1.050283e+07,5855.766389,0,1760.523646,8
8,43.652816,-79.343073,-5.311153e+06,1.050283e+07,5604.462508,0,1170.229135,8
9,43.656542,-79.343635,-5.310553e+06,1.050283e+07,5408.326913,2,599.523934,8
11,43.663994,-79.344760,-5.309353e+06,1.050283e+07,5204.805472,2,706.692568,1
13,43.671448,-79.345886,-5.308153e+06,1.050283e+07,5273.518749,0,1876.006350,1
14,43.675174,-79.346449,-5.307553e+06,1.050283e+07,5408.326913,0,2471.718541,1
16,43.682629,-79.347575,-5.306353e+06,1.050283e+07,5855.766389,2,3667.339360,4
17,43.643150,-79.346109,-5.312653e+06,1.050335e+07,5901.694672,0,2559.756161,8
